In [241]:
train_texts = []


with open("train_sentences.txt", 'r') as f:
    for line in f:
        train_texts.append(line.strip('\n'))

In [242]:
train_nes = []

with open("train_nes.txt", 'r') as f:
    for line in f:
        train_nes.append(line.strip('\n'))

In [243]:
for i, line in enumerate(train_nes):
    splitted = line.split()
    train_nes[i] = []
    for j in range(len(splitted) // 3):
        train_nes[i].append((int(splitted[3 * j]), int(splitted[3 * j + 1]), splitted[3 * j + 2]))


In [244]:
train_nes[:10]

[[(115, 7, 'PERSON'),
  (123, 9, 'PERSON'),
  (134, 8, 'PERSON'),
  (143, 10, 'PERSON'),
  (162, 9, 'PERSON')],
 [],
 [],
 [],
 [],
 [(0, 8, 'ORG'), (9, 13, 'ORG')],
 [(0, 9, 'PERSON'), (18, 8, 'PERSON'), (47, 9, 'ORG')],
 [(35, 9, 'ORG'), (45, 7, 'ORG'), (53, 8, 'PERSON'), (62, 9, 'PERSON')],
 [(126, 13, 'ORG'), (140, 2, 'ORG')],
 [(72, 14, 'ORG'), (89, 10, 'ORG')]]

In [245]:
people = set()
orgs = set()

In [246]:
import glob

for filename in glob.glob('collection3/Collection3/*.ann'):
    with open(filename, 'r') as f:
        for line in f:
            _, tag, name = line.strip().split('\t')
            tag = tag[:3]
            if tag == 'ORG':
                orgs.add(name)
            if tag == 'PER':
                people.add(name)

In [247]:
print(len(orgs), len(people))

2868 5540


In [455]:
from pymystem3 import Mystem

m = Mystem()
text = train_texts[0]

def lemmatize_with_pos(text):
    result = []
    pos = 0
    lemmatized = m.analyze(text.strip())
    names = []
    for dic in lemmatized:
        if 'analysis' in dic:
            if not dic['analysis']:
                result.append((dic['text'], pos, len(dic['text'])))
            else:
                try:
                    tp = dic['analysis'][0]['gr'][2:5]
                    if tp == 'имя' or tp == 'фам':
                        names.append((dic['analysis'][0]['lex'], pos, len(dic['text'])))
                except Exception:
                    print("meow")
                result.append((dic['analysis'][0]['lex'], pos, len(dic['text'])))
        pos += len(dic['text'])
    return result, names
    
print(lemmatize_with_pos(text))


([('выступление', 0, 11), ('в', 12, 1), ('ресторан', 14, 10), ('и', 25, 1), ('кабак', 27, 7), ('между', 37, 5), ('горячий', 43, 7), ('и', 51, 1), ('десерт', 53, 8), ('в', 63, 1), ('мюзик-холл', 65, 12), ('и', 78, 1), ('фешенебельный', 80, 13), ('отель', 94, 6), ('для', 103, 3), ('король', 107, 7), ('густав', 115, 7), ('шведский', 123, 9), ('альфонса', 134, 8), ('испанский', 143, 10), ('принц', 155, 6), ('уэльский', 162, 9), ('для', 173, 3), ('вандербильт', 177, 13), ('и', 191, 1), ('ротшильд', 193, 10)], [('густав', 115, 7), ('альфонса', 134, 8), ('ротшильд', 193, 10)])


In [249]:
text[162:(162+9)]

'Уэльского'

In [250]:
lem_people = []
for per in people:
    lem_people.append(lemmatize_with_pos(per))

In [251]:
lem_orgs = []
for org in orgs:
    leorg = lemmatize_with_pos(org)
    if leorg == []:
        lem_orgs.append((org, 0, len(org)))
    else:
        lem_orgs.append(lemmatize_with_pos(org))

In [49]:
for text in train_texts[:1000]:
    l_text = lemmatize_with_pos(text)
    for i in range(len(l_text)):
        for per in lem_people:
            flag = True
            for j in range(len(per)):
                if l_text[i + j] != per[j]:
                    flag = False
                    break
            if flag:
                print(per)

[('в', 0, 1), ('путин', 2, 5)]
[('юрий', 0, 4)]
[('навальный', 0, 9)]
[('навальный', 0, 9)]
[('си', 0, 2), ('цзиньпина', 3, 8)]
[('лукашенко', 0, 9)]
[('токарев', 0, 7)]
[('виктор', 0, 6)]
[('морозов', 0, 7)]
[('меркель', 0, 7)]
[('руперт', 0, 6), ('мердок', 7, 6)]


In [48]:
for text in train_texts[:100]:
    l_text = lemmatize_with_pos(text)
    for i in range(len(l_text)):
        for org in lem_orgs:
            flag = True
            for j in range(len(org)):
                if l_text[i + j] != org[j]:
                    flag = False
                    break
            if flag:
                print(org)

[('Facebook', 0, 8)]
[('правительство', 0, 13)]


In [252]:
for i, nes in enumerate(train_nes):
    j = 0
    tt = lemmatize_with_pos(train_texts[i])
    dic = {t[1]:t[0] for t in tt}
    while j < len(nes):
        if nes[j][0] not in dic:
            break
        tag = [(dic[nes[j][0]],)]
        while j + 1 < len(nes) and (nes[j][0] + nes[j][1] + 1) == nes[j + 1][0] and nes[j][2] == nes[j + 1][2]:
            j += 1
            if nes[j][0] not in dic:
                break
            tag.append((dic[nes[j][0]],))
        if nes[j][2] == 'ORG':
            lem_orgs.append(tag)
        elif nes[j][2] == 'PERSON':
            lem_people.append(tag)
        j += 1

In [253]:
print(lem_orgs[-100:])

[[('банк',)], [('ростелеком',)], [('государство',)], [('агентство',), ('по',), ('развитие',), ('инновационный',), ('предпринимательство',)], [('руководство',), ('канал',)], [('медиаресурс',)], [('World',), ('Fashion',), ('Channel',)], [('канал',)], [('ведомость',)], [('газпром',), ('межрегионгаз',), ('санкт-петербург',)], [('ИА',), ('Regnum',)], [('кпсс',)], [('ведомость',)], [('роснефть',)], [('BP',)], [('роснефть',)], [('роснефтегаз',)], [('BP',)], [('роснефть',)], [('российский',), ('государственный',), ('нефтяной',), ('компания',)], [('в',), ('контакт',)], [('Caramba',), ('TV',)], [('стс',), ('медиа',)], [('аль-каида',)], [('Vestas',)], [('Vestas',)], [('оон',)], [('оон',)], [('Microsoft',)], [('интернет',), ('Yahoo',)], [('роснефть',)], [('Bank',), ('of',), ('America',), ('Merrill',), ('Lynch',)], [('Bank',), ('of',), ('America',)], [('Citigroup',), ('Global',), ('Markets',), ('Ltd',)], [('газпром-медиа',)], [('интернет',)], [('ICE',)], [('NASDAQ',)], [('NASDAQ',)], [('ICE',)], [(

In [254]:
print(lem_people[-100:])

[[('пастернак',)], [('грегори',), ('чамитофф',)], [('Gregory',), ('Chamitoff',)], [('гаррет',), ('рейсман',)], [('Garrett',), ('Reisman',)], [('виктор',)], [('вертинский',)], [('владимир',), ('путин',)], [('меркель',)], [('чандлер',)], [('мурак',)], [('реджеп',), ('эрдоган',)], [('андрей',), ('терновский',)], [('санников',)], [('тимошенко',)], [('ющенко',)], [('ильф',)], [('петров',)], [('мердок',)], [('бадри',), ('патаркацишвили',)], [('егор',), ('свиридов',)], [('виктор',), ('шендерович',)], [('роман',), ('коноплев',)], [('михаил',), ('мишустин',)], [('форсайт',)], [('роман',), ('коноплев',)], [('нарышкин',)], [('джон',), ('бенер',)], [('михаил',), ('федотов',)], [('александр',), ('хлопонин',)], [('равиль',), ('гайнутдин',)], [('сансерн',), ('каевкамнерд',)], [('петр',), ('порошенко',)], [('артур',), ('чилингаров',)], [('джеймс',), ('миллер',)], [('хоровитц',)], [('ольга',), ('фрейденберг',)], [('игорь',), ('северянин',)], [('иза',), ('кремер',)], [('пьеро',), ('вертинский',)], [('пу

In [262]:
for org in lem_orgs:
    if len(org) == 1 and len(org[0][0]) <= 2:
        if org[0][0] in ['в', 'и', 'с', 'по', 'он', 'на', 'за', 'п', 'т', 'Ъ']:
            lem_orgs.remove(org)


In [263]:
for org in lem_orgs:
    if len(org) == 1 and len(org[0][0]) <= 2:
        print(org)


[('ГД', 0, 2)]
[('ЕК', 0, 2)]
[('цб', 0, 2)]
[('AP', 0, 2)]
[('НП', 0, 2)]
[('GM', 0, 2)]
[('ИА', 0, 2)]
[('кс', 0, 2)]
[('су', 0, 2)]
[('BP', 0, 2)]
[('ск', 0, 2)]
[('GE', 0, 2)]
[('ГУ', 0, 2)]
[('ер', 0, 2)]
[('ер', 1, 2)]
[('РД', 0, 2)]
[('ко', 0, 2)]
[('мк', 0, 2)]
[('цк', 0, 2)]
[('сф', 0, 2)]
[('вр', 0, 2)]
[('ГУ', 0, 2)]
[('сп', 0, 2)]
[('СР', 0, 2)]
[('кп', 0, 2)]
[('се', 0, 2)]
[('ЖЖ', 0, 2)]
[('ан', 0, 2)]
[('ЦС', 0, 2)]
[('оп', 0, 2)]
[('вс', 0, 2)]
[('ГП', 1, 2)]
[('ес',)]
[('BP',)]
[('ми',)]
[('ес',)]
[('BP',)]
[('S',)]
[('RU',)]
[('ес',)]
[('вр',)]
[('вр',)]
[('BP',)]
[('S',)]
[('вр',)]
[('BP',)]
[('ес',)]
[('вс',)]
[('ми',)]
[('ес',)]
[('НК',)]
[('BP',)]
[('HP',)]
[('ес',)]
[('вр',)]
[('ю',)]
[('ес',)]
[('вс',)]
[('ес',)]
[('BP',)]
[('BP',)]
[('U',)]
[('G',)]


In [259]:
for per in lem_people:
    if len(per) == 1 and len(per[0][0]) <= 2:
        if per[0][0] in ['в', 'и', 'с', 'по', 'он', 'на', 'за', 'к', 'н', 'м', 'ли', 'р', 'г']:
            lem_people.remove(per)

In [260]:
for per in lem_people:
    if len(per) == 1 and len(per[0][0]) <= 2:
        print(per)

[('БГ',)]


In [456]:
def get_tagged(text):
    answer = []
    l_text, names = lemmatize_with_pos(text)
    i = 0
    while i < len(l_text):
        found = False
        for per in lem_people:
            flag = True
            for j in range(len(per)):
                if (i + j) >= len(l_text) or l_text[i + j][0] != per[j][0]:
                    flag = False
                    break
            if flag:
                found = True
                answer.append([(l_text[i + j][1], l_text[i + j][2], 'PERSON') for j in range(len(per))])
                i += len(per) - 1
                break
        if found:
            i += 1
            continue
        found = False
        for org in lem_orgs:
            flag = True
            for j in range(len(org)):
                if (i + j) >= len(l_text) or l_text[i + j][0] != org[j][0]:
                    flag = False
                    break
            if flag:
                answer.append([(l_text[i + j][1], l_text[i + j][2], 'ORG') for j in range(len(org))])
                i += len(org) - 1
                found = True
                break
        if found:
            i += 1
            continue
        if (text[l_text[i][1]:(l_text[i][1] + l_text[i][2])].isupper() and l_text[i][2] > 1) or (ord('a') <= ord(l_text[i][0][0].lower()) <= ord('z')):
            answer.append([(l_text[i][1], l_text[i][2], 'ORG')])
        i += 1
    for n in names:
        answer.append([(n[1], n[2], 'PERSON')])
    return answer
                

In [457]:
get_tagged(train_texts[0])

[[(115, 7, 'PERSON'), (123, 9, 'PERSON')],
 [(134, 8, 'PERSON'), (143, 10, 'PERSON')],
 [(162, 9, 'PERSON')],
 [(115, 7, 'PERSON')],
 [(134, 8, 'PERSON')],
 [(193, 10, 'PERSON')]]

In [458]:
def answer_to_string(answer):
    s = ""
    for t in answer:
        for el in t:
            s += f'{str(el[0])} {str(el[1])} {el[2]} '
    s += "EOL"
    return s
    

In [459]:
answer_to_string(get_tagged(train_texts[0]))

'115 7 PERSON 123 9 PERSON 134 8 PERSON 143 10 PERSON 162 9 PERSON 115 7 PERSON 134 8 PERSON 193 10 PERSON EOL'

In [444]:
print(train_texts[0])

Выступления в ресторанах и кабаках — между горячим и десертом; в мюзик-холлах и фешенебельных отелях — для королей Густава Шведского, Альфонса Испанского, принца Уэльского, для Вандербильтов и Ротшильдов.


In [463]:
import traceback
with open('input_17.txt', 'r') as fin, open('output.txt', 'w') as fout:
    for line in fin:
        try:
            print(answer_to_string(get_tagged(line)), file=fout, flush=True)
        except Exception as e:
            print('EOL', file=fout)
            print("meow", e)
            traceback.print_exc()
            print(line)

In [453]:
for org in lem_orgs:
    if len(org) == 1 and org[0][0][:4] == 'клуб':
        print(org)

[('клуб',)]


In [440]:
lem_orgs.remove([('порт', 0, 5)])

In [355]:
len(lem_orgs)

4252